In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer
import unicodedata
import re

In [2]:
# Load the Rotten Tomatoes dataset (adjust the filename if needed)
rt_df = pd.read_csv('RottenTomatoes/rotten_tomatoes_movies.csv')

# Display first few rows
rt_df.head()


rotten_tomatoes_link  \
0                              m/0814255   
1                              m/0878835   
2                                   m/10   
3                 m/1000013-12_angry_men   
4  m/1000079-20000_leagues_under_the_sea   

                                         movie_title  \
0  Percy Jackson & the Olympians: The Lightning T...   
1                                        Please Give   
2                                                 10   
3                    12 Angry Men (Twelve Angry Men)   
4                       20,000 Leagues Under The Sea   

                                          movie_info  \
0  Always trouble-prone, the life of teenager Per...   
1  Kate (Catherine Keener) and her husband Alex (...   
2  A successful, middle-aged Hollywood songwriter...   
3  Following the closing arguments in a murder tr...   
4  In 1866, Professor Pierre M. Aronnax (Paul Luk...   

                                   critics_consensus content_rating  \
0  Though it may seem like just another Harry Pot...             PG   
1  Nicole Holofcener's newest might seem slight i...              R   
2  Blake Edwards' bawdy comedy may not score a pe...              R   
3  Sidney Lumet's feature debut is a superbly wri...             NR   
4  One of Disney's finest live-action adventures,...              G   

                                              genres          directors  \
0  Action & Adventure, Comedy, Drama, Science Fic...     Chris Columbus   
1                                             Comedy  Nicole Holofcener   
2                                    Comedy, Romance      Blake Edwards   
3                                    Classics, Drama       Sidney Lumet   
4           Action & Adventure, Drama, Kids & Family  Richard Fleischer   

                                      authors  \
0  Craig Titley, Chris Columbus, Rick Riordan   
1                           Nicole Holofcener   
2                               Blake Edwards   
3                               Reginald Rose   
4                                 Earl Felton   

                                              actors original_release_date  \
0  Logan Lerman, Brandon T. Jackson, Alexandra Da...            2010-02-12   
1  Catherine Keener, Amanda Peet, Oliver Platt, R...            2010-04-30   
2  Dudley Moore, Bo Derek, Julie Andrews, Robert ...            1979-10-05   
3  Martin Balsam, John Fiedler, Lee J. Cobb, E.G....            1957-04-13   
4  James Mason, Kirk Douglas, Paul Lukas, Peter L...            1954-01-01   

   ...      production_company  tomatometer_status tomatometer_rating  \
0  ...        20th Century Fox              Rotten               49.0   
1  ...  Sony Pictures Classics     Certified-Fresh               87.0   
2  ...             Waner Bros.               Fresh               67.0   
3  ...    Criterion Collection     Certified-Fresh              100.0   
4  ...                  Disney               Fresh               89.0   

  tomatometer_count  audience_status  audience_rating audience_count  \
0             149.0          Spilled             53.0       254421.0   
1             142.0          Upright             64.0        11574.0   
2              24.0          Spilled             53.0        14684.0   
3              54.0          Upright             97.0       105386.0   
4              27.0          Upright             74.0        68918.0   

   tomatometer_top_critics_count  tomatometer_fresh_critics_count  \
0                             43                               73   
1                             44                              123   
2                              2                               16   
3                              6                               54   
4                              5                               24   

   tomatometer_rotten_critics_count  
0                                76  
1                                19  
2                                 8  


In [3]:
# Create an explicit copy to avoid SettingWithCopyWarning
rt_df_cleaned = rt_df[[
    "movie_title", "genres",
    "tomatometer_rating", "tomatometer_count",
    "audience_rating", "audience_count",
    "tomatometer_fresh_critics_count", "tomatometer_rotten_critics_count"
]].copy()


In [4]:
# Convert columns to numeric safely
rt_df_cleaned.loc[:, "tomatometer_rating"] = pd.to_numeric(rt_df_cleaned["tomatometer_rating"], errors="coerce")
rt_df_cleaned.loc[:, "audience_rating"] = pd.to_numeric(rt_df_cleaned["audience_rating"], errors="coerce")
rt_df_cleaned.loc[:, "tomatometer_count"] = pd.to_numeric(rt_df_cleaned["tomatometer_count"], errors="coerce").fillna(0).astype(int)
rt_df_cleaned.loc[:, "audience_count"] = pd.to_numeric(rt_df_cleaned["audience_count"], errors="coerce").fillna(0).astype(int)

rt_df_cleaned["tomatometer_rating"] = rt_df_cleaned["tomatometer_rating"].fillna(rt_df_cleaned["tomatometer_rating"].median())
rt_df_cleaned["audience_rating"] = rt_df_cleaned["audience_rating"].fillna(rt_df_cleaned["audience_rating"].median())

# Fill missing genres
rt_df_cleaned["genres"] = rt_df_cleaned["genres"].fillna("Unknown")

In [7]:
# Standardize movie title function
# def clean_movie_title(title):
#     if pd.isna(title):
#         return ""
#     title = str(title).strip().lower()  # Convert to lowercase and remove leading/trailing spaces
#     title = unicodedata.normalize('NFKD', title).encode('ASCII', 'ignore').decode('utf-8')  # Remove accents
#     title = re.sub(r'[^a-zA-Z0-9\s]', '', title)  # Remove special characters
#     return title
def normalize_text(text):
    if pd.isna(text):  # Handle missing values
        return ""
    text = str(text)
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')  # Remove accents
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    return text.strip()

rt_df_cleaned["movieTitle"] = rt_df_cleaned["movie_title"].apply(normalize_text)

# Drop the 'movie_title' column
rt_df_cleaned = rt_df_cleaned.drop(columns=["movie_title"])

# Move 'movieTitle' to the first column position
cols = list(rt_df_cleaned.columns)
cols.insert(0, cols.pop(cols.index("movieTitle")))
rt_df_cleaned = rt_df_cleaned[cols]

In [8]:
rt_df_cleaned.head()

movieTitle  \
0  Percy Jackson  the Olympians The Lightning Thief   
1                                       Please Give   
2                                                10   
3                     12 Angry Men Twelve Angry Men   
4                       20000 Leagues Under The Sea   

                                              genres  tomatometer_rating  \
0  Action & Adventure, Comedy, Drama, Science Fic...                49.0   
1                                             Comedy                87.0   
2                                    Comedy, Romance                67.0   
3                                    Classics, Drama               100.0   
4           Action & Adventure, Drama, Kids & Family                89.0   

   tomatometer_count  audience_rating  audience_count  \
0              149.0             53.0        254421.0   
1              142.0             64.0         11574.0   
2               24.0             53.0         14684.0   
3               54.0             97.0        105386.0   
4               27.0             74.0         68918.0   

   tomatometer_fresh_critics_count  tomatometer_rotten_critics_count  
0                               73                                76  
1                              123                                19  
2                               16                                 8  
3                               54                                 0  
4                               24                                 3

In [10]:
rt_df_cleaned.to_csv("cleaned/rottentomatoMovies2.csv", index=False)

In [9]:
print(rt_df_cleaned.dtypes)

movieTitle                           object
genres                               object
tomatometer_rating                  float64
tomatometer_count                   float64
audience_rating                     float64
audience_count                      float64
tomatometer_fresh_critics_count       int64
tomatometer_rotten_critics_count      int64
dtype: object
